# Who is heading for Diabetes?

This is the predictive part of the 2017 Melbourne Datathon.

The task is to predict the probability that a patient will be dispensed a drug related to Diabetes post 2015. This is quite important research as it will be an early warning system for doctors so intervention can potentially be made before it is too late.

Use the patients that we have provided all the records for to build your model, then see how it performs on these unseen people.

For patient ID'S 279,201 to 558,352 you need to submit a file with 2 columns, the Patient_ID and the probability in the range [0-1]. The file will have 279,153 rows including the header row. An example submission file is provided for download.

In [38]:
import tqdm

import pandas as pd
import numpy as np
import sqlite3

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import normalize, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

## Data retrieval functions

In [5]:
conn = sqlite3.connect("../../sql/datasci.db")

In [6]:
def patient_data(conn, patient_id):
    """
    Return the patient data.
    """
    SQL = """
SELECT *
FROM transactions a
LEFT OUTER JOIN ChronicIllness_LookUp b 
    ON a.Drug_ID = b.MasterProductID 
LEFT OUTER JOIN patients c
    ON a.Patient_ID = c.Patient_ID
LEFT OUTER JOIN classification d
    ON a.Patient_ID = d.Patient_ID
LEFT OUTER JOIN social e
    ON c.postcode = e.postcode
WHERE a.Patient_ID = {}
AND a.prescription_week < '2016-01-01'
ORDER BY prescription_week
    """.format(patient_id)

    return pd.read_sql_query(SQL, conn)

In [7]:
df = patient_data(conn, 100)
df.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,SourceSystem_Code,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,IsDeferredScript,...,year_of_birth,postcode,Patient_ID,Target,postcode,advantage_score,disadvantage_score,economic_score,occupation_score,population
0,100,2415,32470,4867,F,2010-10-17,2011-06-19,LIPI3,8215J,0,...,1938,3500,100,0,3500,907,914,912,929,30659
1,100,2415,32470,9430,F,2011-01-02,2011-02-13,ZOMI1,8266C,0,...,1938,3500,100,0,3500,907,914,912,929,30659
2,100,2415,32470,9430,F,2011-04-03,2011-04-17,ZOMI1,8266C,0,...,1938,3500,100,0,3500,907,914,912,929,30659
3,100,2002,0,9430,F,2011-04-03,2011-05-08,ZOMI1,8266C,0,...,1938,3500,100,0,3500,907,914,912,929,30659
4,100,2415,1637,9430,F,2011-06-05,2011-06-12,ZOMI1,8266C,0,...,1938,3500,100,0,3500,907,914,912,929,30659


## Feature extraction functions

In [83]:
gender_map = {'F': 1, 'M': 0, 'U': 0.5}

def gender(patient_data):
    return gender_map[patient_data.gender[0]]

def age(patient_data):
    patient_age = 2016 - patient_data.year_of_birth[0]
    if patient_age > 100: 
        return 50 
    else: 
        return patient_age
        
def socio_score(patient_data):
    return patient_data.disadvantage_score[0] or 1000.

def had_diabetes(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Diabetes').any())

def had_lipids(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Lipids').any())

def had_hypertension(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Hypertension').any())

## Compute some basic features of the data 

In [85]:
def feature_extract(patient_frame):
    """
    The thing that forms a feature vector.
    
    ** Make sure to partition out data from the 2016 period.
    
    """
    
    x = [gender(patient_frame), 
         age(patient_frame), 
         socio_score(patient_frame),
         had_diabetes(patient_frame),
         had_lipids(patient_frame),
         had_hypertension(patient_frame)]
    
    y = patient_frame.Target[0]
    
    return x, y


## Create the matrix of features and vector of targets

In [86]:
n = 10000

X, Y = [], []
for i in tqdm.tqdm_notebook(np.random.randint(0, 200000, n)): 
    x, y = feature_extract(patient_data(conn, i))
    X.append(x)
    Y.append(y)
    
X = np.vstack(X)
y = np.array(Y)

## Look at the descriptive power of the features.


Look at the relationship of variables and plot them.

## Try out a set of different classifiers

Note: the feature matrix is usually transformed to have zero mean and unit standard deviation.

In [87]:
X = StandardScaler().fit_transform(X)

Partition the data into test and train datasets.

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [89]:
classifiers = [
    ('Random Forrest', RandomForestClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Adaboost',AdaBoostClassifier() ),
    ('SVM',SVC(probability=True))]

for name, clf in classifiers:
    print('Classifier: {}'.format(name))
    model = clf.fit(X_train, y_train)
    y_true, y_pred = y_test, clf.predict(X_test) 
    print(classification_report(y_true, y_pred))


Classifier: Random Forrest
             precision    recall  f1-score   support

          0       0.95      0.94      0.95      4115
          1       0.74      0.76      0.75       885

avg / total       0.91      0.91      0.91      5000

Classifier: Decision Tree
             precision    recall  f1-score   support

          0       0.94      0.94      0.94      4115
          1       0.73      0.71      0.72       885

avg / total       0.90      0.90      0.90      5000

Classifier: Adaboost
             precision    recall  f1-score   support

          0       0.98      0.94      0.96      4115
          1       0.75      0.91      0.82       885

avg / total       0.94      0.93      0.93      5000

Classifier: SVM
             precision    recall  f1-score   support

          0       0.99      0.93      0.96      4115
          1       0.75      0.94      0.84       885

avg / total       0.94      0.93      0.94      5000



## Form a submission

In [90]:
submission = pd.read_csv('../../submissions/diabetes_submission_example.csv')

In [ ]:
for i in tqdm.tqdm_notebook(range(submission.shape[0])):
    
    submission_X, submission_y = feature_extract(patient_data(conn, submission.Patient_ID[i]))
    
    submission_X = np.array(submission_X).reshape(1, -1)
    
    submission['Diabetes'][row] = clf.predict_proba(submission_X)[0][0]